In [2]:
pip install langchain langgraph composio-langgraph langchain_groq

  Using cached langgraph-0.5.3-py3-none-any.whl.metadata (6.9 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 501.2/501.2 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [26]:
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from typing import Annotated
from langchain_groq import ChatGroq
from composio_langgraph import Action, ComposioToolSet
from langgraph.graph import StateGraph, START
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables.config import RunnableConfig

In [4]:
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]

In [5]:
from google.colab import userdata
groq_api_key = userdata.get('GROQ_API_KEY')

llm = ChatGroq( model="meta-llama/llama-4-scout-17b-16e-instruct", temperature=0, api_key= groq_api_key)

In [18]:
composio_api_key = userdata.get('COMPOSIO_API_KEY')

In [20]:
def get_composio_tools():
    toolset = ComposioToolSet(composio_api_key)
    tools = toolset.get_tools(actions=[
        Action.COMPOSIO_SEARCH_TAVILY_SEARCH,
        Action.GOOGLEDOCS_CREATE_DOCUMENT_MARKDOWN
    ])
    return tools

In [37]:
!composio add googledocs

> Redirecting you to the login page
> https://app.composio.dev/?cliKey=d7f7aa03-4aff-4bf7-b686-9a59a88ac403
> Enter authentication code: 
Aborted!


In [21]:
tools = get_composio_tools()
llm_with_tools = llm.bind_tools(tools)

def agent_node(state):
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

tool_node = tools

INFO:composio.utils.shared:Actions cache is outdated, refreshing cache...


In [22]:
system_message = """
You are a sophisticated research assistant. Perform comprehensive research on the given query and provide detailed analysis. Focus on:
- Key concepts and main ideas
- Current developments and trends
- Important stakeholders and their roles
- Relevant data and statistics
- Critical analysis and implications
-  When answering a question, if you need to search the web, call the COMPOSIO_SEARCH_TAVILY_SEARCH tool.
- token limit 1000
Use the tool like this:
<function_call>
{
  "query": "AI-powered diagnostic tools adoption in hospitals by 2030",
  "search_depth": "basic",
  "include_images": false,
  "include_raw_content": false,
  "max_results": 5
}
</function_call>

Do not use quotes around booleans or integers. Only call the function if you genuinely need a web search.
Create a detailed report on the research and write it in Google Docs.
Ensure all information is accurate, up-to-date, and properly sourced. Present findings in a clear, structured format suitable for professional analysis.
"""

In [24]:
def build_graph():
    builder = StateGraph(GraphState)
    builder.add_node("agent", agent_node)
    builder.add_node("tools", ToolNode(tools=tools))
    builder.add_conditional_edges("agent", tools_condition)
    builder.add_edge("tools", "agent")
    builder.add_edge(START, "agent")
    return builder

In [36]:
print("🧠 Deep Research Agent")

# Input from user
topic = input("Enter your research topic: ").strip()
domain = input("Enter the domain (e.g., Health, Technology, etc.): ").strip()

if topic and domain:
    print("\n🔄 Generating research questions...")

    # Step 1: Generate questions
    question_prompt = (
        f"Generate exactly 3 specific yes/no research questions about the topic '{topic}' "
        f"in the domain '{domain}'. Respond ONLY with a numbered list and NOTHING ELSE."
    )
    q_messages = [{"role": "user", "content": question_prompt}]
    response = llm.invoke(q_messages)
    questions = [q.strip() for q in response.content.split('\n') if q.strip()]

    print("\n📌 Research Questions")
    for i, q in enumerate(questions, 1):
        print(f"{i}. {q}")

    # Step 2: Research each question
    answers = []
    for i, question in enumerate(questions):
        print(f"\n🔍 Researching: {question}")
        memory = MemorySaver()
        graph = build_graph().compile(checkpointer=memory)
        config = typing.cast(RunnableConfig, {"configurable": {"thread_id": str(i+1)}})

        messages = [{"role": "user", "content": f"{system_message}\n\nResearch question: {question}"}]
        answer = ""
        for chunk in graph.stream({"messages": messages}, config=config, stream_mode="values"):
            content = chunk["messages"][-1].content
            answer += content + "\n"
        answers.append((question, answer))
        print("✅ Done")

    # Step 3: Compile answers into a report
    print("\n🧾 Compiling final report...")
    final_memory = MemorySaver()
    final_graph = build_graph().compile(checkpointer=final_memory)
    final_config = typing.cast(RunnableConfig, {"configurable": {"thread_id": "final"}})

    qa_html = "\n".join(
        f"<h2>{i+1}. {q}</h2><p>{a}</p>" for i, (q, a) in enumerate(answers)
    )

    report_prompt = (
        f"You are a sophisticated research assistant. token limit 5000. "
        f"Compile the following research findings into a professional, McKinsey-style report using HTML:\n\n"
        f"Topic: {topic}\nDomain: {domain}\n\n"
        f"Research Findings:\n{qa_html}\n\n"
        f"Create a Google Doc with this full report using GOOGLEDOCS_CREATE_DOCUMENT_MARKDOWN."
    )

    messages = [{"role": "user", "content": report_prompt}]
    report_output = ""
    for chunk in final_graph.stream({"messages": messages}, config=final_config, stream_mode="values"):
        content = chunk["messages"][-1].content
        report_output += content + "\n"

    print("\n✅ Final Report:\n")
    print(report_output)

    # Step 4: Follow-up question
    followup = input("\nAsk a follow-up question (or press Enter to skip): ").strip()
    if followup:
        followup_msgs = [{"role": "user", "content": followup}]
        for event in final_graph.stream({"messages": followup_msgs}, config=final_config, stream_mode="values"):
            print(event["messages"][-1].content)


🧠 Deep Research Agent
Enter your research topic: ai in blockchain in 2026
Enter the domain (e.g., Health, Technology, etc.): tech

🔄 Generating research questions...

📌 Research Questions
1. 1. Will the integration of AI in blockchain technology by 2026 lead to a significant reduction in transaction processing times?
2. 2. Will AI-powered smart contracts be widely adopted in blockchain networks by 2026?
3. 3. Will AI-driven security measures in blockchain prevent more than 90% of cyber attacks on blockchain networks by 2026?

🔍 Researching: 1. Will the integration of AI in blockchain technology by 2026 lead to a significant reduction in transaction processing times?
✅ Done

🔍 Researching: 2. Will AI-powered smart contracts be widely adopted in blockchain networks by 2026?
✅ Done

🔍 Researching: 3. Will AI-driven security measures in blockchain prevent more than 90% of cyber attacks on blockchain networks by 2026?
✅ Done

🧾 Compiling final report...

✅ Final Report:

You are a sophistic